# Introduction
In notebook 42 and 45 I made sure that every Fatty acid chain needed was produced in our model. Now, as we will apply the restrictive approach to our model, I will combine them all into a generic acyl-chain that reflects the distribution identified i Tang et al. (2009). The generic acyl chain will then need to have a chemical formula that matches the distribution given so that we maintain mass balance.

Finally, this chain will be attached to the phospholipid heads, and these metabolites modified so that mass balance is achieved.

Finally, we will add the fatty acid requirement to biomass. __How much??__ Stick to 7% g/g? 

In [69]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction, Metabolite

In [70]:
model_e_coli = cameo.load_model('iML1515')

In [71]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

In [72]:
#add general Acyl-ACP reaction
model.add_metabolites(Metabolite(id='acylACP_c', name = 'Generic Acyl-ACP fatty acid', compartment = 'c', charge = -1, formula = 'C27.07H51.14O8N2PRS'))

Note: the chemical formula of the generic acyl ACP is calculated by considering the chemical formula of each individual FA, and weighing it with the stoichiometry that the data provides.

In [73]:
#add reaction forming generic Acyl-ACP
model.add_reaction(Reaction(id='ACYLACP', name = 'Pseudoreaction to create general acyl-ACP chain'))

In [74]:
model.reactions.ACYLACP.annotation['sbo'] = 'SBO:0000631'

In [75]:
model.reactions.ACYLACP.add_metabolites({
    model.metabolites.get_by_id('2mtridecACP_c'):-0.004,
    model.metabolites.myrsACP_c:-0.025,
    model.metabolites.get_by_id('3mtdecACP_c'):-0.096,
    model.metabolites.get_by_id('2mtdecACP_c'):-0.006,
    model.metabolites.get_by_id('pdecACP_c'):-0.027,
    model.metabolites.get_by_id('2mpdecACP_c'): -0.279,
    model.metabolites.get_by_id('palmACP_c'): -0.347,
    model.metabolites.get_by_id('3mhdecACP_c'): -0.114,
    model.metabolites.get_by_id('2mhdecACP_c'): -0.037,
    model.metabolites.hdecACP_c:-0.024,
    model.metabolites.ocdcaACP_c:-0.041,
    model.metabolites.acylACP_c:1
})

In [76]:
model.reactions.ACYLACP.check_mass_balance()

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

27.07 is not an integer (in formula C27.07H51.14O8N2PRS)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:104: UserWarning:

51.14 is not an integer (in formula C27.07H51.14O8N2PRS)



{'C': -3.552713678800501e-15}

Note, here we don't have complete mass balance because to define the generic acyl-ACP (and acyl-CoA) we need to have decimals in the chemical formula. But, as the imbalance here is sooo small it is oke and we will continue on with it.

In [77]:
#save&commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

### acylcoa_c  should be the acyl-acp chemical formula but then add C10H12N5O9P2 to the chemical formula and remove the R

In [41]:
#convert the acylACP to acyl-CoA metabolite.
#add acyl-CoA metabolite first
model.add_metabolites(Metabolite(id='acylcoa_c', name = 'Generic Acyl-CoA', compartment = 'c', charge = -3, formula = 'C37.07H63.14O17N7P3S'))

In [44]:
model.metabolites.acylcoa_c.annotation['sbo'] = 'SBO:0000247'

In [47]:
#add reaction to convert the two
model.add_reaction(Reaction(id='ACYAC', name= 'CoA:[acyl-carrier-protein] transferase'))

In [49]:
model.reactions.ACYAC.bounds = (-1000,1000)

In [ ]:
model.reactions.ACYAC.add_metabolites({
    model.metabolites.ACP_c:-1,
    model.metabolites.acylcoa_c:-1,
    model.metabolites.coa_c:1,
    model.metabolites.acylACP_c:1,
    model.metabolites.h_c:1
})

In [ ]:
model.reactions.ACYAC.check_mass_balance()

When we have the generic acyl-coa molecule made, we can the incorporate it into the glycerolipid heads and ensure the three lipids we need for the biomass reaction can be made. 

The acyl-Coa chains are incorporated into the sn-glycerol-3-phosphate molecule, in two steps to give a 1,2-diacyl-glycerol-3-phosphate. this can then react further into the three glycerolipids we need: Phosphatidylglycerol, Diphosphatidylglycerol and Phosphatidylethanolamine. 

To incorporate the chains, we just need to change the chemical formula of certain metabolites and change some reactions to accomodate the generic acyl-chains.

In [61]:
#start in reaction ACCOAACT: change the acoa to the generic acylcoa_c met I made
model.reactions.ACCOAACT.add_metabolites({
    model.metabolites.acoa_c:1,
    model.metabolites.acylcoa_c:-1
})

Reaction identifier,ACCOAACT
Name,acyl-CoA:sn-glycerol-3-phosphate 1-O-acyltransferase
Memory address,0x028591d70a88
Stoichiometry,acoa_c + glyc3p_c <=> aglyc3p_c + coa_c Acyl coenzyme A + sn-Glycerol <=> 1-Acyl-sn-glycerol 3-phosphate + CoA
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [ ]:
#change formula of aglyc3p_c
model.metabolites.aglyc3p_c.formula = 

chemical formula should lose C21H32N7O16P3S to give plain acyl chain.
then add up the sn-glycerol to the chem formula to givve the formula of aglyc3p.

In [ ]:
#check mass balance
model.reactions.ACCOAACT.check_mass_balance()

In [62]:
#do the same for the ACCOATT reaction
model.reactions.ACCOATT.add_metabolites({
    model.metabolites.acoa_c:1,
    model.metabolites.acylcoa_c:-1
})

Reaction identifier,ACCOATT
Name,acyl-CoA:1-acyl-sn-glycerol-3-phosphate 2-O-acyltransferase
Memory address,0x028591d74308
Stoichiometry,acoa_c + aglyc3p_c <=> coa_c + pa_c Acyl coenzyme A + 1-Acyl-sn-glycerol 3-phosphate <=> CoA + Phosphatidate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [ ]:
#change formula of pa_c
model.metabolites.pa_c.formula = 

Here you just add the acylcoa_c formula - coa (C21H32N7O16P3S) to the formula of aglyc3p. to give pa_c formula.

In [ ]:
#check mass balance
model.reactions.ACCOATT.check_mass_balance()

In [ ]:
#then change the cdpdag_c  chemical formula accordingly
model.metabolites.cdpdag_c.formula = 

to pa_c you add the chemical formula of CDP, and then subtract that of the ppi. 

In [ ]:
#then continue like this through until all lipids have been made

In [65]:
model.metabolites.pa_c.summary()

In [ ]:
#remove acoa_c metabolite to prevent duplicate generic acyl chains being present
model.remove_metabolites(model.metabolites.acoa_c)

To do:

    - convert to general acyl-COA
    - incorporate this into the glycerol-molecules
    - add to biomass rct